#setup
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#read previous behavior data pickles and edit behavior events

In [ ]:
# output_dir = '/content/drive/MyDrive/behaviors_220915night_nbs_WT_xp0/'
output_dir = '/content/drive/MyDrive/behaviors_220918night_GiDREADD/'
video_name = 'PZ71_1'
end_frame = 36000*3.5
start_frame = 0

behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
behavior = pickle.load(behavior_file)
behavior_file.close()

behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
behavior_events = pickle.load(behavior_events_file)
behavior_events_file.close()

print(video_name)
first_20_min = behavior.truncate(before=start_frame, after=(end_frame - 1)).copy()
male_side_seconds = (first_20_min[first_20_min["behavior"] == 'male_side']["behavior"].count() + first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count()) / 30
male_side_percent = male_side_seconds / (end_frame / 30)
mount_seconds = first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count() / 30

first_20_min_events = behavior_events.copy()
male_side_entrances = first_20_min_events[first_20_min_events["behavior_events"] == 'male_side']["behavior_events"].count()
mount_events = first_20_min_events[first_20_min_events["behavior_events"] == 'mount']["behavior_events"].count()

print('male_side seconds: ', male_side_seconds)
print('male_side percent: ', male_side_percent)
print('mount seconds: ', mount_seconds)
print('mount events: ', mount_events)

PZ71_1
male_side seconds:  735.1666666666666
male_side percent:  0.17503968253968252
mount seconds:  55.233333333333334
mount events:  26


In [ ]:
print(first_20_min_events[first_20_min_events["behavior_events"] == 'mount'])

        behavior
300  female_side
301  female_side
302  female_side
303  female_side
304  female_side
..           ...
495  female_side
496  female_side
497  female_side
498  female_side
499  female_side

[200 rows x 1 columns]


In [ ]:
def change_behavior_event(behavior, behavior_events, event_index, new_event):
  behavior_events_i = behavior_events.loc[behavior_events['events_index'] == event_index]
  behavior_events["behavior_events"][behavior_events_i] = new_event
  start_index = behavior_events["events_index"][behavior_events_i]
  end_index = behavior_events["events_index"][behavior_events_i + 1]
  behavior["behavior"][start_index:end_index]

In [ ]:
event_index = 1
new_event = 'female_side'
change_behavior_event(behavior, behavior_events, new_event)